# Classification d'images en Deep Learning - Classificateur binaire

Ceci est notre première tentative de classification de terrains en Deep Learning. Afin de pouvoir reconnaître nos terrains uniquement à partir d'images de ceux-ci, il est nécessaire de construire des [OCC ("One-Class Classifier")](https://en.wikipedia.org/wiki/One-class_classification).

### Entraîner un classificateur d'images

1. Charger et normaliser nos ensembles de données d'entraînement et de test en utilisant *torchvision*.
2. Définir un réseau neural convolutif (CNN)
3. Définir une fonction de perte
4. Former le réseau sur les données relatives à l'entraînement
5. Vérifier la validité du réseau sur les données de test

**1. Charger et normaliser nos ensembles de données**

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

%matplotlib inline

EPOCHS = 10

Les ensembles de données *torchvision* sont des images PILImage de portée [0, 1].
Nous les transformons en Tenseurs de portée normalisée [-1, 1].

In [ ]:
trainset = torchvision.datasets.ImageFolder(root="data/classifier/training",
                           transform=transforms.Compose([
                               transforms.Resize(32),
                               transforms.CenterCrop(32),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))

trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.ImageFolder(root="data/classifier/test",
                           transform=transforms.Compose([
                               transforms.Resize(32),
                               transforms.CenterCrop(32),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))

testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('voiture', 'terrain')

Voici quelques images de l'ensemble d'entraînement avec leur classe associée.



In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

**2. Définir un réseau neural convolutif (CNN)**

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

**3. Définir une fonction de perte et un optimisateur**

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

**4. Entraîner le réseau**

In [ ]:
for epoch in range(EPOCHS):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

Sauvegarde de notre réseau, on peut ensuite le recharger via:
```py
net = Net()
net.load_state_dict(torch.load('models/classifier.pt'))
```

In [ ]:
torch.save(net.state_dict(), 'models/classifier.pt')

**5. Vérifier la validité du réseau sur les données de test**

Nous avons formé le réseau à l'aide de l'ensemble des données de formation mais nous devons vérifier si le réseau a appris quelque chose. Nous pouvons le faire en lui donnant une image et en vérifiant que la classe qu'il a associé à cette image est correcte. 

Voici quelques images de l'ensemble de test avec leur classe associée.

In [ ]:
dataiter = iter(testloader)
dataiter.next()
dataiter.next()
dataiter.next()
images, labels = dataiter.next()

# print images
imshow(torchvision.utils.make_grid(images))
print('Classe: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

On peut maintenant voir quelles sont les prédictions de notre classificateur pour les images ci-dessus:


In [ ]:
outputs = net(images)

_, predicted = torch.max(outputs, 1)

print('Prédictions: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(4)))

Les résultats semblent bons.
Voyons maintenant les performances de notre réseau sur tout l'ensemble de données de test:

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Précision par classe:


In [ ]:
class_correct = list(0. for i in range(2))
class_total = list(0. for i in range(2))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(2):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(2):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))